<a href="https://colab.research.google.com/github/kyjness/10th_Seminar/blob/main/catboost_1%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
y!pip install catboost

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 데이터 로드 (훈련 데이터와 테스트 데이터)
train_data = pd.read_csv('/train.csv')  # Price 포함된 훈련 데이터
test_data = pd.read_csv("/test.csv")    # Price 제거된 테스트 데이터

# 훈련 데이터 확인
print("🔹 훈련 데이터 크기:", train_data.shape)
print("🔹 테스트 데이터 크기:", test_data.shape)

# 범주형 변수 결측값 -> Unknown으로 대체
cat_cols = ['Brand', 'Material', 'Size', 'Style', 'Color', 'Laptop Compartment', 'Waterproof']
train_data[cat_cols] = train_data[cat_cols].fillna("Unknown")
test_data[cat_cols] = test_data[cat_cols].fillna("Unknown")
#수치형 변수 결측값 -> median으로 대체
num_cols1 = ['Compartments', 'Weight Capacity (kg)', 'Price']
num_cols2 = ['Compartments', 'Weight Capacity (kg)']
train_data[num_cols1] = train_data[num_cols1].fillna(train_data[num_cols1].median())
test_data[num_cols2] = test_data[num_cols2].fillna(test_data[num_cols2].median())


# 범주형 변수 리스트
cat_cols = ['Brand', 'Material', 'Size', 'Style', 'Color', 'Laptop Compartment', 'Waterproof']

# 범주형 변수를 category 타입으로 변경 (CatBoost에서 자동 인식)
for col in cat_cols:
    train_data[col] = train_data[col].astype("category")
    test_data[col] = test_data[col].astype("category")

# Feature (X) / Target (y) 분리
X = train_data.drop(columns=["Price"])
y = train_data["Price"]

# Train / Validation 데이터 분할
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

print("✅ 데이터 전처리 완료!")

🔹 훈련 데이터 크기: (300000, 11)
🔹 테스트 데이터 크기: (200000, 10)
✅ 데이터 전처리 완료!


In [ ]:
# CatBoost 모델 정의 및 학습
model = CatBoostRegressor(
    iterations=2000,   # 트리 개수 (증가 가능)
    learning_rate=0.01,  # 학습률
    depth=8,  # 트리 깊이 (6~10 추천)
    cat_features=cat_cols,  # 범주형 변수 전달
    loss_function='RMSE',  # 손실 함수
    eval_metric='RMSE',  # 평가 지표
    verbose=100,  # 학습 과정 출력
    early_stopping_rounds=50  # 과적합 방지
)

# 모델 학습
model.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=50)

print("✅ CatBoost 모델 학습 완료!")

0:	learn: 39.0627102	test: 38.9439199	best: 38.9439199 (0)	total: 502ms	remaining: 16m 42s
100:	learn: 39.0344511	test: 38.9222107	best: 38.9222107 (100)	total: 37.3s	remaining: 11m 41s
200:	learn: 39.0206442	test: 38.9155885	best: 38.9155885 (200)	total: 1m 11s	remaining: 10m 36s
300:	learn: 39.0110355	test: 38.9135133	best: 38.9135112 (299)	total: 1m 47s	remaining: 10m 9s
400:	learn: 39.0036618	test: 38.9125334	best: 38.9125252 (398)	total: 2m 22s	remaining: 9m 27s
500:	learn: 38.9976052	test: 38.9119542	best: 38.9119287 (495)	total: 2m 55s	remaining: 8m 45s
600:	learn: 38.9904206	test: 38.9113185	best: 38.9113185 (600)	total: 3m 31s	remaining: 8m 11s
700:	learn: 38.9846814	test: 38.9111472	best: 38.9111069 (691)	total: 4m 5s	remaining: 7m 35s
800:	learn: 38.9785108	test: 38.9108649	best: 38.9108649 (800)	total: 4m 42s	remaining: 7m 3s
900:	learn: 38.9726620	test: 38.9103013	best: 38.9103013 (900)	total: 5m 17s	remaining: 6m 27s
1000:	learn: 38.9654103	test: 38.9098761	best: 38.90985

#catboost 성능 개선 원할 시
	•	iterations=1000 → 2000으로 증가 (더 많은 학습)
	•	depth=6 → 8로 증가 (더 복잡한 패턴 학습)
	•	learning_rate=0.05 → 0.01로 조정 (더 세밀한 학습)
	•	early_stopping_rounds=50을 조정 (과적합 방지)

In [ ]:
# 검증 데이터 예측
y_pred_valid = model.predict(X_valid)

# 성능 평가 지표 계산
rmse = np.sqrt(mean_squared_error(y_valid, y_pred_valid))
mae = mean_absolute_error(y_valid, y_pred_valid)
r2 = r2_score(y_valid, y_pred_valid)

print(f"📊 모델 성능 평가:")
print(f"🔹 RMSE (Root Mean Squared Error): {rmse:.4f}")
print(f"🔹 MAE (Mean Absolute Error): {mae:.4f}")
print(f"🔹 R² (R-Squared): {r2:.4f}")

📊 모델 성능 평가:
🔹 RMSE (Root Mean Squared Error): 38.9099
🔹 MAE (Mean Absolute Error): 33.6373
🔹 R² (R-Squared): 0.0017


In [ ]:
# 테스트 데이터 예측
test_predictions = model.predict(test_data)

# 예측 결과 저장
test_data["Predicted_Price"] = test_predictions
test_data.to_csv("test_predictions.csv", index=False)

print("✅ 테스트 데이터 예측 완료! 결과가 test_predictions.csv에 저장되었습니다.")

✅ 테스트 데이터 예측 완료! 결과가 test_predictions.csv에 저장되었습니다.


In [ ]:
import pandas as pd

# ID를 30000부터 시작하도록 재설정
submission = pd.DataFrame({
    "id": range(300000, 300000 + len(test_predictions)),  # 30000번부터 시작하는 ID 생성
    "Price": test_predictions  # 예측된 가격
})

# 소수점 3자리까지 반올림
submission["Price"] = submission["Price"].round(3)

# CSV 파일로 저장 (컴퓨터에 저장)
file_path = "submission.csv"
submission.to_csv(file_path, index=False)

# 저장된 파일 확인
print("✅ 예측 결과가 'submission.csv'로 저장되었습니다!")
print(submission.head(10))  # 상위 10개 샘플 확인

from google.colab import files
files.download("submission.csv")

✅ 예측 결과가 'submission.csv'로 저장되었습니다!
       id   Price
0  300000  82.338
1  300001  82.263
2  300002  82.052
3  300003  82.218
4  300004  81.150
5  300005  82.669
6  300006  82.385
7  300007  82.883
8  300008  83.165
9  300009  81.546


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>